In [34]:
import nbformat
import sys
import textwrap
import xlrd
filename='sample_data.xlsx'
data=xlrd.open_workbook(filename)
data_sheet=data.sheet_by_index(0)



for i in range(1,data_sheet.nrows):
    nb = nbformat.v4.new_notebook()
    
    
    gse=data_sheet.cell(i,0).value
    gse=gse.encode('ascii','ignore')
    
    platform=data_sheet.cell(i,1).value
    platform=platform.encode('ascii','ignore')
    
    sample_id=gse+"_"+platform
    
    control_samples_excel=data_sheet.cell(i,2).value.encode('ascii','ignore')
    experimental_samples_excel=data_sheet.cell(i,3).value.encode('ascii','ignore')
    
    number_of_samples=len(control_samples_excel.split("|"))
      

    source_0 = textwrap.dedent("""\
        from IPython.display import HTML
        HTML('''<script>
        code_show=true; 
        function code_toggle() {
         if (code_show){
         $('div.input').hide();
         } else {
         $('div.input').show();
         }
         code_show = !code_show
        } 
        $( document ).ready(code_toggle);
        </script>
        <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')       
        """)
    code_cell_0 = nbformat.v4.new_code_cell(source=source_0)
    nb.cells.append(code_cell_0)
    
   
    
        
    markdown_cell_1_content = "## Exploratory analysis of dataset" + " " + gse + " " + "obtained from Gene Expression Omnibus (GEO)"
    markdown_cell_1 = nbformat.v4.new_markdown_cell(markdown_cell_1_content)
    nb.cells.append(markdown_cell_1)

    markdown_cell_2_content = "The data discussed in this publication have been deposited in NCBI's Gene Expression Omnibus and are accessible through GEO Series accession number" + " " + gse+ " " + "and are available [here](at https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=)."

    markdown_cell_2 = nbformat.v4.new_markdown_cell(markdown_cell_2_content)
    nb.cells.append(markdown_cell_2)

    markdown_cell_3_content = textwrap.dedent("""\
        ### Goals of this report
        This report aims to conduct an exploratory analysis of the RNA-seq data generated by this study using an [RNA-seq pipeline](https://f1000researchdata.s3.amazonaws.com/manuscripts/9804/8fa2f4e7-d0f3-488c-85f1-0b3a9816faef_9110_-_avi_maaayn.pdf?doi=10.12688/f1000research.9110.1) developed by [the Ma'ayan Lab](http://labs.icahn.mssm.edu/maayanlab/) at [the Icahn School of Medicine at Mount Sinai](http://icahn.mssm.edu/). This [Jupyter Notebook](http://jupyter.org/) was automatically generated <a href='#Citations'>[1]</a>.
        """)
    markdown_cell_3 = nbformat.v4.new_markdown_cell(markdown_cell_3_content)
    nb.cells.append(markdown_cell_3)

    markdown_cell_4_content = textwrap.dedent("""\
        ### Analysis Overview
        #### 1.  <a href='#Dimensionality Reduction'>Dimensionality Reduction</a> 
        Dimensionality reduction helps analyze and visualize multidimensional data by reducing the data to 3D space. Principal component analysis (PCA) was applied to the data in this analysis.
        #### 2.  <a href='#Interactive Heatmap'>Interactive Heatmap</a>
        [Clustergrammer](http://clustergrammer.readthedocs.io/clustergrammer_widget.html) is an interactive tool that can be used to visualize a wide range of biological data. It is particularly useful for gene expression data. Clusters of upregulated and downregulated genes for the groups are clearly visible, which simplifies further analysis and comparisons between differential expression signatures.
        #### 3.  <a href='#Differential Expression Analysis'>Differential Expression Analysis</a>
        This analysis helps to identify genes that show differences in expression as compared to control. This information can be used to obtain a deeper understanding of the molecular mechanisms of biological processes.
        #### 4.  <a href='#Gene Set Enrichment Analysis'>Gene Set Enrichment Analysis</a>
        Gene Set Enrichment Analysis uses the upregulated and downregulated genes identified in differential expression analysis to provide valuable insight into the underlying mechanisms of biological processes.
        #### 5.  <a href='#Small Molecule Predictions'>Small Molecule Predictions</a>
        This analysis produces a set of small molecules that are predicted either mimic or reverse the expression profiles observed in the experiment.

        """)
    markdown_cell_4 = nbformat.v4.new_markdown_cell(markdown_cell_4_content)
    nb.cells.append(markdown_cell_4)

    markdown_cell_5 = nbformat.v4.new_markdown_cell("<a id='Dimensionality Reduction'></a>")
    nb.cells.append(markdown_cell_5)

    markdown_cell_6_content = textwrap.dedent("""\
        ## 1. Dimensionality Reduction

        ### Principal Component Analysis (PCA)
        PCA was used to reduce the data to three-dimensional space.Counts per million (CPM) and z-score normalization methods were applied to the data. A PCA plot is expected to show clusters of samples relevant to experimental groups <a href='#Citations'>[2]</a>.
        """)
    markdown_cell_6 = nbformat.v4.new_markdown_cell(markdown_cell_6_content)
    nb.cells.append(markdown_cell_6)

    import textwrap

    
    source_variables=textwrap.dedent("""\

    #%store -r gse
    #%store -r platform
    #%store -r control_samples_excel
    #%store -r experimental_samples_excel
    #%store -r number_of_samples
    #%store -r sample_id

    gse="{gse}"
    platform="{platform}"
    sample_id="{sample_id}"
    control_samples_excel="{control_samples_excel}"
    experimental_samples_excel="{experimental_samples_excel}"


    """.format(**locals()))


    code_cell_variables=nbformat.v4.new_code_cell(source=source_variables)
    nb.cells.append(code_cell_variables)


    source_number_of_samples="number_of_samples=({})".format(number_of_samples)

    code_cell_number_of_samples=nbformat.v4.new_code_cell(source=source_number_of_samples)
    nb.cells.append(code_cell_number_of_samples)
    
    
    
    
    source_1 = textwrap.dedent("""\
        %%capture

        # Initialize Plotly and R magic
        %load_ext rpy2.ipython

        %R require(limma)
        %R require(edgeR)
        %R require(ggpot)
        %R require(gridExtra)
        from plotly.offline import init_notebook_mode
        init_notebook_mode()

        import sys
        import xlrd
        sys.path.append('scripts')
        sys.path.append('gloriascripts')
        import pandas as pd
        import archs4
        import numpy as np 
        
        
    
        
        
        #platform=platform_nb
        #control_samples_excel=control_samples_excel_nb
        #experimental_samples_excel=experimental_samples_excel_nb
        #number_of_samples=number_of_samples_nb
        #sample_id=sample_id_nb
        
        #sample_id
        #platform
        #control_samples_excel
        #experimental_samples_excel
        #number_of_samples
        
        """)

    code_cell_1 = nbformat.v4.new_code_cell(source=source_1)
    nb.cells.append(code_cell_1)

    source_10="import xlrd"
    code_cell_10 = nbformat.v4.new_code_cell(source=source_10)
    nb.cells.append(code_cell_10)


#     source_4 = "rawcount_dataframe,sample_title_dict,treatment_dataframe = archs4.fetch_dataset('{}')".format(sample_id)

#     code_cell_4 = nbformat.v4.new_code_cell(source=source_4)
#     nb.cells.append(code_cell_4)

    source_4 = "rawcount_dataframe,sample_title_dict,treatment_dataframe = archs4.fetch_dataset(sample_id)"

    code_cell_4 = nbformat.v4.new_code_cell(source=source_4)
    nb.cells.append(code_cell_4)



    source_5 = textwrap.dedent("""\
    
        rawcount_dataframe=rawcount_dataframe.drop(rawcount_dataframe.index[rawcount_dataframe.mean(axis=1)<=rawcount_dataframe.shape[1]])
        rawcount_dataframe_original=rawcount_dataframe.copy(deep=True)

        group_dataframe=pd.DataFrame()

        group_dataframe_list=sample_title_dict.values()

        group_dataframe['treatment']=pd.Series(sample_title_dict.get(v) for v in rawcount_dataframe_original.columns)
        group_dataframe['treatment']=pd.Series(v.split(' (GSM')[0] for v in group_dataframe['treatment'])
        treatment_dataframe.columns=['treatment']

        group_dataframe.index=treatment_dataframe['treatment']
        treatment_dataframe.index=group_dataframe['treatment']

        rawcount_dataframe.columns=group_dataframe['treatment']


        norm_dataframe=pd.DataFrame()
        norm_dataframe=rawcount_dataframe.copy(deep=True)



        #cpm normalizing the columns

        cpm_factors=rawcount_dataframe.sum(axis=0)/1000000
        for i in range(0,rawcount_dataframe.shape[1]):
            norm_dataframe.iloc[:,i]=rawcount_dataframe.iloc[:,i]/cpm_factors[i]

        #z score normalize the rows

        from scipy import stats
        for rows in range(0,rawcount_dataframe.shape[0]):
            norm_dataframe.iloc[rows,:] = stats.zscore(rawcount_dataframe.iloc[rows,:])

        from sklearn.decomposition import PCA
        pca_3=PCA(n_components=3)
        pca_3.fit_transform(norm_dataframe)

        import plot_3d_scatter
        fig=plot_3d_scatter.plot_3d_scatter(pca_3.components_[0],pca_3.components_[1],pca_3.components_[2], size=5, color_by=treatment_dataframe['treatment'].tolist(),
        xlab= 'PC1 ({:.2%} of variance explained)'.format(pca_3.explained_variance_ratio_[0]),
        ylab= 'PC2 ({:.2%} of variance explained)'.format(pca_3.explained_variance_ratio_[1]),
        zlab= 'PC3 ({:.2%}of variance explained)'.format(pca_3.explained_variance_ratio_[2]))


        """)

    code_cell_5 = nbformat.v4.new_code_cell(source=source_5)
    nb.cells.append(code_cell_5)


   # Plotting PCA

    source_2 = textwrap.dedent("""\
            import plotly.offline as py
            from plotly.offline import iplot,init_notebook_mode
            init_notebook_mode()
            py.iplot(fig)
        """)

    code_cell_2 = nbformat.v4.new_code_cell(source=source_2)
    nb.cells.append(code_cell_2)

    source_5 = textwrap.dedent("""\

        %Rpush rawcount_dataframe
        %Rpush treatment_dataframe
        %Rpush group_dataframe
        %R design<-model.matrix(~1+treatment, group_dataframe)
        %R voom_dataframe<-voom(rawcount_dataframe,plot=FALSE)
        %R voom_dataframe<-as.data.frame(voom_dataframe)
        %Rpull voom_dataframe
        """)

    code_cell_5 = nbformat.v4.new_code_cell(source=source_5)
    nb.cells.append(code_cell_5)


    markdown_cell_9_content = textwrap.dedent("""\
        #### Figure 1 | PCA analysis of the RNA-seq dataset. 
        The plot depicts the PCA analysis of the dataset. Each point on the graph represents a single sample. The axis represent the principal components, which are linear combinations of the variables from each expression vector. The variance accounted for by each principal component can be seen on the respective axis.
        """)
    markdown_cell_9 = nbformat.v4.new_markdown_cell(markdown_cell_9_content)
    nb.cells.append(markdown_cell_9)


    #Clustergrammer

    markdown_cell_7 = nbformat.v4.new_markdown_cell("<a id='Interactive Heatmap'></a>")
    nb.cells.append(markdown_cell_7)

    markdown_cell_8_content = textwrap.dedent("""\
    ## 2. Interactive Heatmap

    500 most variable genes were visualized in this [Clustergrammer](http://clustergrammer.readthedocs.io/clustergrammer_widget.html) <a href='#Citations'>[3]</a>.The names of the samples can be seen as column labels; you can zoom in to see the gene names as rows. The tool is interactive; you can arrange row and column orders by using the panel to the left of the heatmap. The cluster of interest can be selected and the resulting genes can be sent to [Enrichr](http://amp.pharm.mssm.edu/Enrichr/) for further analysis. The clustergrammer instance can be shared and the matrix can be downloaded as a .txt file.
   
    """)
    markdown_cell_8 = nbformat.v4.new_markdown_cell(markdown_cell_8_content)
    nb.cells.append(markdown_cell_8)

    source_3 = textwrap.dedent("""\
        import plot_clustergrammer
        plot_clustergrammer.display(voom_dataframe) 
        """)

    code_cell_3 = nbformat.v4.new_code_cell(source=source_3)
    nb.cells.append(code_cell_3)

    markdown_cell_5_content = textwrap.dedent("""\
        #### Figure 2 | Interactive heatmap for top 500 most variable genes.
        The clustergrammer depicts the normalized expression levels of top 500 most variable genes. Voom method in R was used to normalize the data. The columns represent the sample names. You can zoom in to see the gene names as rows.
        """)
    markdown_cell_5 = nbformat.v4.new_markdown_cell(markdown_cell_5_content)
    nb.cells.append(markdown_cell_5)

    markdown_cell_12 = nbformat.v4.new_markdown_cell("<a id='Gene Co-Expression Analysis'></a>")
    nb.cells.append(markdown_cell_12)


    #EXAMPLE
    #source_9 = "data=xlrd.open_workbook('{}')".format(filename)


    source_6 = textwrap.dedent("""\
        from rpy2.robjects import r, pandas2ri
        import pandas as pd
        import sys

        r.source('scripts/signature.R')
        pandas2ri.activate()
    """)
    code_cell_6 = nbformat.v4.new_code_cell(source=source_6)
    nb.cells.append(code_cell_6)

    source_ctrl="control_samples=list(control_samples_excel.split('|')[v] for v in range(number_of_samples))"

    code_cell_ctrl=nbformat.v4.new_code_cell(source=source_ctrl)
    nb.cells.append(code_cell_ctrl)


    source_exp="experimental_samples=list(experimental_samples_excel.split('|')[v] for v in range(number_of_samples) )"

    code_cell_exp=nbformat.v4.new_code_cell(source=source_exp)
    nb.cells.append(code_cell_exp)



    source_20= textwrap.dedent("""\
        import gene_signature
        limma_dataframe=gene_signature.signature(rawcount_dataframe_original,method = 'limma', experimental_samples=experimental_samples, control_samples=control_samples)
        limma_dataframe['significant']=[rowData['adj.P.Val'] < 0.2 and abs(rowData['logFC']) > 1 for index, rowData in limma_dataframe.iterrows()]

        sign=limma_dataframe[limma_dataframe['significant']==True]
        insign=limma_dataframe[limma_dataframe['significant']==False]

        import numpy as np
        import customplot
        """)

    code_cell_20 = nbformat.v4.new_code_cell(source=source_20)
    nb.cells.append(code_cell_20)


    #Differential Expression Analysis

    markdown_cell_18 = nbformat.v4.new_markdown_cell("<a id='Differential Expression Analysis'></a>")
    nb.cells.append(markdown_cell_18)

    markdown_cell_17_content = textwrap.dedent("""\
         ## 3. Differential Expression Analysis
        """)
    markdown_cell_17 = nbformat.v4.new_markdown_cell(markdown_cell_17_content)
    nb.cells.append(markdown_cell_17)

    markdown_cell_13_content = textwrap.dedent("""\
        ### 3.1 Volcano and MA Plots

        [Limma package](https://bioconductor.org/packages/release/bioc/html/limma.html) in R was used to obtain differential gene expression signatures <a href='#Citations'>[4]</a>. 

        A volcano plot visualizes the results of gene expression analysis and depicts the logarithm of P-values (-log10(P-value)) on the y-axis versus log-transformed fold changes (logFC) on the x-axis.

        MA plot depicts the log-transformed fold changes (M) for each gene versus its average expression (A). Fold changes, P-values and average expressions were calculated using limma package in R.

        """)

    markdown_cell_13 = nbformat.v4.new_markdown_cell(markdown_cell_13_content)
    nb.cells.append(markdown_cell_13)

    source_18=textwrap.dedent("""\

    volcano_df=df_sign_up = pd.DataFrame({'logFC': limma_dataframe['logFC'],'adj.P.Val':limma_dataframe['adj.P.Val'],'p.Value':limma_dataframe['P.Value'],'gene':limma_dataframe.index,'AveExpr' : limma_dataframe['AveExpr']})

    """)

    code_cell_18 = nbformat.v4.new_code_cell(source=source_18)
    nb.cells.append(code_cell_18)


    source_7 = textwrap.dedent("""\
       %%R -i volcano_df -w 900 -h 480 -u px
        library(ggplot2)
        library(ggrepel)
        volcano_df<-as.data.frame(volcano_df)
        Genes<-as.factor(abs(volcano_df$logFC) > 1 & volcano_df$adj.P.Val<0.2)
        levels(Genes) <- sub("^TRUE$", "Significant",levels(Genes))
        levels(Genes) <- sub("^FALSE$", "Not Significant",levels(Genes))
        g_volcano <-ggplot(data=volcano_df, aes(x=logFC, y=-log10(p.Value)),na.rm=TRUE) + geom_point(aes(col=Genes),size=4) + scale_color_manual(values=c("black", "red"))+ theme(panel.background = element_blank()) + xlab('Log-tranformed fold changes') + ylab('-log10 (P-Value)') + theme_bw()+theme(axis.text=element_text(size=12),axis.title=element_text(size=14)) + guides(fill=guide_legend(title=NULL)) +ggtitle("Volcano Plot")+theme(plot.title = element_text(family="bold", size=18,hjust=0.5))
        set.seed(42)
        g_ma<-ggplot(data=volcano_df, aes(x=AveExpr, y=logFC)) + geom_point(aes(col=Genes),size=4) + scale_color_manual(values=c("black", "red"))+ theme(panel.background = element_blank())+ xlab('Average gene expression') + ylab('Log-tranformed fold changes') + theme_bw()+theme(axis.text=element_text(size=12),axis.title=element_text(size=14)) + guides(fill=guide_legend(title=NULL)) + ggtitle("MA Plot") +theme(plot.title = element_text(family="bold", size=18,hjust = 0.5))

        grid.arrange(g_volcano, g_ma, ncol=2)

        """)
    code_cell_7 = nbformat.v4.new_code_cell(source=source_7)
    nb.cells.append(code_cell_7)


    markdown_cell_14_content = textwrap.dedent("""\
        #### Figure 3 | Volcano and MA plots for the RNA-seq dataset.  
        Volcano plot depicts the log of P-value for each gene verus log-transformed fold changes as compared to the control samples. MA plot depicts the log-transformed fold changes (M) for each gene versus its average expression (A). Fold changes, average expressions and P-values were calculated using limma package in R. Limma method along with the thresholds (|logFC| > 1 and adjusted P-value < 0.2) were used to identify the significant genes.
        """)
    markdown_cell_14 = nbformat.v4.new_markdown_cell(markdown_cell_14_content)
    nb.cells.append(markdown_cell_14)



    markdown_cell_19_content = textwrap.dedent("""\
         ### 3.2 Further exploration of significant genes identified by limma

         The following tables show significant upregulated genes and downregulated genes identifies by limma. You can scroll and search for genes of interest by using the toolbar in the tables.
        """)
    markdown_cell_19 = nbformat.v4.new_markdown_cell(markdown_cell_19_content)
    nb.cells.append(markdown_cell_19)

    source_12 = textwrap.dedent("""\
        list_sign_up=limma_dataframe[(limma_dataframe['logFC']>0) & (limma_dataframe['significant']==True)].index
        list_sign_dn=limma_dataframe[(limma_dataframe['logFC']<0) & (limma_dataframe['significant']==True)].index

        df_sign_up = pd.DataFrame({'Significant Upregulated Genes':list_sign_up, 'Log-transformed fold changes': limma_dataframe.loc[list_sign_up]['logFC'],'Adjusted P-value':limma_dataframe.loc[list_sign_up]['adj.P.Val']})
        df_sign_up=df_sign_up[['Significant Upregulated Genes','Log-transformed fold changes','Adjusted P-value']]


        df_sign_dn = pd.DataFrame({'Significant Downregulated Genes':list_sign_dn, 'Log-transformed fold changes': limma_dataframe.loc[list_sign_dn]['logFC'],'Adjusted P-value':limma_dataframe.loc[list_sign_dn]['adj.P.Val']})
        df_sign_dn=df_sign_dn[['Significant Downregulated Genes','Log-transformed fold changes','Adjusted P-value']]

        import qgrid
        qgrid.nbinstall(overwrite=True)
        qgrid.set_defaults(show_toolbar=True, remote_js=True,export_mode=True)       
        """)
    code_cell_12 = nbformat.v4.new_code_cell(source=source_12)
    nb.cells.append(code_cell_12)



    source_13=textwrap.dedent("""\
    df_sign_up.index=range(1,len(df_sign_up)+1)
    qgrid.show_grid(df_sign_up) 
    """)
    code_cell_13 = nbformat.v4.new_code_cell(source=source_13)
    nb.cells.append(code_cell_13)

    markdown_cell_20_content = textwrap.dedent("""\
         #### Table 1 | Significant upregulated genes identified by limma as compared to control samples
        The table shows the list of significant upregulated genes as compared to control samples. Limma method in R along with the thresholds (logFC > 1 and adjusted P-value < 0.2) were used to identify the significant genes.
        """)
    markdown_cell_20 = nbformat.v4.new_markdown_cell(markdown_cell_20_content)
    nb.cells.append(markdown_cell_20)




    source_14=textwrap.dedent("""\
    df_sign_dn.index=range(1,len(df_sign_dn)+1)
    qgrid.show_grid(df_sign_dn) 
    """)
    code_cell_14 = nbformat.v4.new_code_cell(source=source_14)
    nb.cells.append(code_cell_14)

    markdown_cell_21_content = textwrap.dedent("""\
         #### Table 2 | Significant downregulated genes identified by limma as compared to control samples
        The table shows the list of significant downregulated genes as compared to control samples. Limma method in R along with the thresholds (logFC < -1 and adjusted P-value < 0.2) were used to identify the significant genes.
        """)
    markdown_cell_21 = nbformat.v4.new_markdown_cell(markdown_cell_21_content)
    nb.cells.append(markdown_cell_21)

    markdown_cell_26=nbformat.v4.new_markdown_cell("<a id='Gene Set Enrichment Analysis'></a>")
    nb.cells.append(markdown_cell_26)


    #Gene Set Enrichment Analysis


    markdown_cell_22_content = textwrap.dedent("""\
        ## 4. Gene Set Enrichment Analysis

        The differential gene expression signatures computed with limma are now used for enrichment analysis on top 500 upregulated and downregulated genes using [Enrichr](http://amp.pharm.mssm.edu/Enrichr/). 

        [Enrichr](http://amp.pharm.mssm.edu/Enrichr/) is a comprehensive gene set enrichment analysis tool. Containing 180,184 annotated gene sets from 102 gene set libraries, it allows for easy and effective search and provides interactive visualizations of enrichment results <a href='#Citations'>[5]</a>. 

        """)
    markdown_cell_22 = nbformat.v4.new_markdown_cell(markdown_cell_22_content)
    nb.cells.append(markdown_cell_22)



    source_cd_1=textwrap.dedent("""\
    limma_up=limma_dataframe.loc[limma_dataframe['logFC']>=0]
    limma_dn=limma_dataframe.loc[limma_dataframe['logFC']<0]
    top_500=limma_up['adj.P.Val'].sort_values(ascending=True)[:500].index
    bottom_500=limma_dn['adj.P.Val'].sort_values(ascending=True).tail(500).index

    #top_500_var_indices=rawcount_dataframe.var(axis=1).sort_values(ascending=False)[:100].index
    #top_500_var_genes=rawcount_dataframe.loc[top_500_var_indices]

    #top_variable_genes=rawcount_dataframe_original.loc[rawcount_dataframe_original.apply(np.var,1).sort_values(ascending=False)[:5000].index]

    #cd_dataframe=gene_signature.signature(top_variable_genes,method = 'CD',experimental_samples = experimental_samples, control_samples = control_samples)

    #cd_dataframe.sort_values('CD',axis=0,ascending=False)[:200].index
    #top_500=cd_dataframe.sort_values('CD',axis=0,ascending=False)[:200].index
    #bottom_500=cd_dataframe.sort_values('CD',axis=0,ascending=False).tail(200).index


    """)
    code_cell_cd_1=nbformat.v4.new_code_cell(source=source_cd_1)
    nb.cells.append(code_cell_cd_1)

    source_cd_2=textwrap.dedent("""\

    import upload_genesets
    dict_up=upload_genesets.enrichr(top_500)[1]
    dict_dn=upload_genesets.enrichr(bottom_500)[1]
    user_list_id_up = dict_up.get('userListId')
    user_list_id_dn=dict_dn.get('userListId')

    import enrichr_library
    #up_chea,dn_chea=enrichr_library.data(gene_set_library='ChEA_2016',user_list_id_up=user_list_id_up,user_list_id_dn=user_list_id_dn)
    #up_bio_process,dn_bio_process=enrichr_library.data(gene_set_library='GO_Biological_Process_2017b',user_list_id_up=user_list_id_up,user_list_id_dn=user_list_id_dn)
    #up_kegg,dn_kegg=enrichr_library.data(gene_set_library='KEGG_2016',user_list_id_up=user_list_id_up,user_list_id_dn=user_list_id_dn)

    enrichr_url_up=upload_genesets.enrichr(top_500)[0]
    enrichr_url_dn=upload_genesets.enrichr(bottom_500)[0]
    """)
    code_cell_cd_2=nbformat.v4.new_code_cell(source=source_cd_2)
    nb.cells.append(code_cell_cd_2)

    source_17=textwrap.dedent("""\
    def make_clickable(val):
        return '<a href="{}">{}</a>'.format(val,val)
    df = pd.DataFrame([enrichr_url_up,enrichr_url_dn])
    df.index=['Upregulated genes','Downregulated genes']
    df.columns=['Links to Enrichr results']
    df.style.format(make_clickable)
    """)
    code_cell_17 = nbformat.v4.new_code_cell(source=source_17)
    nb.cells.append(code_cell_17)


    markdown_cell_23_content = textwrap.dedent("""\
         #### Table 3 | Links to  Enrichr results
        The table includes the links to  Enrichr results for top 500 upregulated and downregulated genes identified by limma.
        """)
    markdown_cell_23 = nbformat.v4.new_markdown_cell(markdown_cell_23_content)
    nb.cells.append(markdown_cell_23)


    markdown_cell_29=nbformat.v4.new_markdown_cell("<a id='Small Molecule Predictions'></a>")
    nb.cells.append(markdown_cell_29)

    #Small molecule predictions


    markdown_cell_24_content = textwrap.dedent("""\
         ### 5. Small Molecule Predictions

        [L1000CDS<sup>2</sup>](http://amp.pharm.mssm.edu/L1000CDS2/#/index) is a state-of-the-art web-based search engine, which provides prioritization of thousands of small-molecule signatures, and their pairwise combination, predicted to either mimic or reverse an input gene expression signature <a href='#Citations'> [6]</a>. It integrates the library of intergrated network-based cellular signatures (LINCS) L1000 data set processed with [Characteristic Direction Method](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-15-79) <a href='#Citations'> [7]</a>.It currently includes information about more than a million gene expression profiles of chemically perturbed human cell lines. L1000CDS<sup>2</sup> was used to identify small molecules that would mimic or reverse the signature, using differential gene expression signatures computed using [Characteristic Direction Method](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-15-79).

        """)
    markdown_cell_24 = nbformat.v4.new_markdown_cell(markdown_cell_24_content)
    nb.cells.append(markdown_cell_24)


    source_16=textwrap.dedent("""\

    import retrieve
    smq_mimic=retrieve.small_molecule_query(top_500,bottom_500, mimic=True)[0]
    smq_reverse=retrieve.small_molecule_query(top_500,bottom_500, mimic=False)[0]

    df = pd.DataFrame([smq_mimic,smq_reverse])

    df.index=['Small molecules predicted to mimic the signature','Small molecules predicted to reverse the signature']
    df.columns=['Links to Small Molecule Query results']
    def make_clickable(val):
        return '<a href="{}">{}</a>'.format(val,val)
    df.style.format(make_clickable)
    """)
    code_cell_16 = nbformat.v4.new_code_cell(source=source_16)
    nb.cells.append(code_cell_16)

    markdown_cell_25_content = textwrap.dedent("""\
        #### Table 4| Links to  Small Molecule Query results
        The table includes the links to Small Molecule Query results. The resulting small molecules are reported, along with the cell line perturbed, dosage and time of exposure.
        """)
    markdown_cell_25 = nbformat.v4.new_markdown_cell(markdown_cell_25_content)
    nb.cells.append(markdown_cell_25) 



    source_19=textwrap.dedent("""\
    
    output=pd.DataFrame()
    output_mimic=retrieve.small_molecule_query(top_500,bottom_500, mimic=True)[1].get('topMeta')
    output_reverse=retrieve.small_molecule_query(top_500,bottom_500, mimic=False)[1].get('topMeta')


    pert_id_mimic = [item.get('pert_id') for item in output_mimic]  
    pert_desc_mimic = [item.get('pert_desc') for item in output_mimic]   

    pert_id_reverse = [item.get('pert_id') for item in output_reverse]  
    pert_desc_reverse = [item.get('pert_desc') for item in output_reverse ]  
    pert_overlap_reverse = [item.get('score') for item in output_reverse]
    pert_overlap_mimic= [item.get('score') for item in output_mimic]

    df_pert_desc_reverse = pd.DataFrame({'Small molecule description':pert_desc_reverse,'Small molecule ID': pert_id_reverse,'Overlap score':pert_overlap_reverse})
    df_pert_desc_reverse = df_pert_desc_reverse[['Small molecule description','Small molecule ID','Overlap score']]


    df_pert_desc_mimic = pd.DataFrame({'Small molecule description':pert_desc_mimic,'Small molecule ID': pert_id_mimic,'Overlap score':pert_overlap_mimic})
    df_pert_desc_mimic = df_pert_desc_reverse[['Small molecule description','Small molecule ID','Overlap score']]
    qgrid.show_grid(df_pert_desc_mimic)
    """)
    code_cell_19 = nbformat.v4.new_code_cell(source=source_19)
    nb.cells.append(code_cell_19)

    markdown_cell_20_content = textwrap.dedent("""\
         #### Table 5 | Small Molecules predicted to mimic the input gene expression signature
        The table shows the list of small molecules that are predicted to mimic the input gene expression signature
        """)
    markdown_cell_20 = nbformat.v4.new_markdown_cell(markdown_cell_20_content)
    nb.cells.append(markdown_cell_20)



    source_21=textwrap.dedent("""\
    df_pert_desc_reverse.index=range(1,len(df_pert_desc_reverse)+1)
    qgrid.show_grid(df_pert_desc_reverse)
    
    """)
    code_cell_21 = nbformat.v4.new_code_cell(source=source_21)
    nb.cells.append(code_cell_21)

    markdown_cell_30_content = textwrap.dedent("""\
         #### Table 6 | Small Molecules predicted to reverse the imput gene expression signature
        The table shows the list of small molecules that are predicted to reverse the input gene expression signature
        """)
    markdown_cell_30 = nbformat.v4.new_markdown_cell(markdown_cell_30_content)
    nb.cells.append(markdown_cell_30)
    
    
    markdown_cell_18 = nbformat.v4.new_markdown_cell("<a id='Citations'></a>")
    nb.cells.append(markdown_cell_18)  

    markdown_cell_citations=("### Citations")
    
    markdown_cell_citations=nbformat.v4.new_markdown_cell(markdown_cell_citations)
    nb.cells.append(markdown_cell_citations)
    
    markdown_cell_28_content = textwrap.dedent("""\
    
    1.  Perez, F.,  Granger, B. E. (2015). Project Jupyter: Computational narratives as the engine of collaborative data science. Technical Report. Technical report, Project Jupyter. 
    2.  Clark, N. R.,  Ma'ayan, A. (2011). Introduction to statistical methods to analyze large data sets: Principal components analysis. Science signaling, 4(190), tr3.
    3.  Fernandez, N. F., Gundersen, G. W., Rahman, A., Grimes, M. L., Rikova, K., Hornbeck, P.,  Ma'ayan, A. (2017). Clustergrammer, a web-based heatmap visualization and analysis tool for high-dimensional biological data. Scientific Data, 4, sdata2017151.
    4.  Smyth, G. K., Ritchie, M., Thorne, N., Wettenhall, J., Shi, W., Hu, Y. (2002). Limma: Linear Models for Microarray and RNA-Seq Data User's Guide.
    5.  Kuleshov MV, Jones MR, Rouillard AD, et al. Enrichr: a comprehensive gene set enrichment analysis web server 2016 update. Nucleic Acids Research. 2016;44(Web Server issue):W90-W97. doi:10.1093/nar/gkw377.
    6.  Duan, Q. L1000CDS2: LINCS L1000 Characteristic Direction Signatures Search Engine. npj Systems Biology and Applications. 2016.
    7.  Clark, N. R., Hu, K. S., Feldmann, A. S., Kou, Y., Chen, E. Y., Duan, Q.,  Ma'ayan, A. (2014). The characteristic direction: a geometrical approach to identify differentially expressed genes. BMC bioinformatics, 15(1), 79.""")
    
    markdown_cell_28 = nbformat.v4.new_markdown_cell(markdown_cell_28_content)
    nb.cells.append(markdown_cell_28)


    # executing and saving the notebook
    from nbconvert.preprocessors.execute import ExecutePreprocessor

    pp = ExecutePreprocessor()
    pp.timeout = 30  # seconds
    pp.interrupt_on_timeout = True

    nb_executed, resources = pp.preprocess(nb, resources={})

    nb_name = "Automated_Notebook" + "_" + sample_id + "." + "ipynb"

    with open(nb_name, 'w') as f:
        f.write(nbformat.v4.writes_json(nb_executed))

    
    
    